## Importaciones y configuración inicial

En esta sección se cargan las librerías necesarias para el modelado, procesamiento de datos y visualización.
Se configura también el dispositivo de cómputo (CPU o GPU) y algunos parámetros generales.

El modelo LSTM se implementará en **PyTorch**, utilizando `DataLoader` para manejar las secuencias temporales.


In [9]:
# --- Librerías base ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- PyTorch ---
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset

import os

# --- Optimización CPU ---
torch.set_num_threads(os.cpu_count())       # Usa todos los hilos disponibles
torch.set_num_interop_threads(os.cpu_count())
print(f"🧠 CPU Threads: {torch.get_num_threads()}")

# --- Configuración ---
plt.style.use("seaborn-v0_8-whitegrid")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📦 Usando dispositivo: {device}")


🧠 CPU Threads: 16
📦 Usando dispositivo: cpu


## Carga y exploración del dataset

Se carga el conjunto de datos sincronizado `ruido_cuenca_sync.csv`, que contiene los registros de ruido en diferentes puntos de la ciudad.  
El objetivo es preparar estos datos para ser utilizados en la red LSTM.

En este paso se:
- Lee el archivo CSV.
- Convierte la columna temporal a formato `datetime`.
- Ordena los datos cronológicamente.


In [10]:
# Cargar el dataset
data_path = "../../data/clean/ruido_cuenca_sync.csv"
df = pd.read_csv(data_path)

# Mostrar info básica
print("Forma del dataset:", df.shape)
display(df.head())

# Ver columnas y verificar si existe 'timestamp' o similar
print(df.columns)

# Si tiene timestamp, conviértelo a datetime
if "timestamp" in df.columns:
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.sort_values("timestamp")


Forma del dataset: (249119, 9)


,Unnamed: 0,ruido_SCP01,ruido_SCP06,ruido_SCP07,ruido_SCP08,ruido_SCP09,ruido_SCP13,ruido_SCP16,ruido_SCP17
0,2025-04-16 00:01:00,41.920000,57.83,59.58,68.250000,75.68,64.780,67.27,67.7
1,2025-04-16 00:02:00,41.931818,57.83,59.58,68.250000,75.68,64.780,67.27,67.7
2,2025-04-16 00:03:00,41.943636,57.83,59.58,68.250000,75.68,64.780,67.27,67.7
3,2025-04-16 00:04:00,41.955455,57.83,59.58,67.742727,75.68,64.780,67.27,67.7
4,2025-04-16 00:05:00,41.967273,57.83,59.58,67.235455,75.68,64.806,67.27,67.7


Index(['Unnamed: 0', 'ruido_SCP01', 'ruido_SCP06', 'ruido_SCP07',
       'ruido_SCP08', 'ruido_SCP09', 'ruido_SCP13', 'ruido_SCP16',
       'ruido_SCP17'],
      dtype='object')


## Preparación de las secuencias temporales

El modelo LSTM requiere entradas con estructura **(muestras, pasos de tiempo, características)**.

En esta sección se:
- Seleccionan las variables numéricas relevantes.
- Normalizan los valores con `MinMaxScaler`.
- Crean las secuencias temporales (ventanas deslizantes).
- Dividen los datos en conjuntos de entrenamiento y prueba.


In [11]:
# Seleccionar solo las columnas numéricas (nivel de ruido, temperatura, etc.)
features = df.select_dtypes(include=[np.number]).columns.tolist()
print("Características numéricas:", features)

# Normalizar datos
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df[features])

# Convertir a tensor
data_scaled = torch.tensor(data_scaled, dtype=torch.float32)

# Definir función para crear ventanas temporales
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length][0])  # Asumimos que la primera columna es el valor objetivo (ruido)
    return torch.stack(X), torch.stack(y)

seq_length = 24  # 24 horas o pasos temporales
X, y = create_sequences(data_scaled, seq_length)

print("Forma de X:", X.shape)
print("Forma de y:", y.shape)

# Dividir en entrenamiento y prueba (80/20)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

# --- Carga de datos optimizada ---
num_workers = os.cpu_count() - 1  # usa todos los núcleos menos 1
batch_size = 128                  # o 256 si tienes RAM suficiente

train_loader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    persistent_workers=True
)

print(f"DataLoader configurado con batch_size={batch_size} y num_workers={num_workers}")



Características numéricas: ['ruido_SCP01', 'ruido_SCP06', 'ruido_SCP07', 'ruido_SCP08', 'ruido_SCP09', 'ruido_SCP13', 'ruido_SCP16', 'ruido_SCP17']
Forma de X: torch.Size([249095, 24, 8])
Forma de y: torch.Size([249095])
DataLoader configurado con batch_size=128 y num_workers=15


## Definición del modelo LSTM

Aquí se implementa el modelo LSTM en PyTorch.  
El modelo cuenta con:
- Capas recurrentes para capturar dependencias temporales.
- Capas densas finales para predecir el nivel de ruido.

Estructura básica:
- Entrada: secuencias de características normalizadas.
- Salida: valor de ruido esperado para el siguiente instante.


In [12]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # solo el último paso
        out = self.fc(out)
        return out

# Crear modelo
input_size = X.shape[2]
model = LSTMModel(input_size=input_size).to(device)
print(model)


LSTMModel(
  (lstm): LSTM(8, 64, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


## Entrenamiento del modelo

Se entrena el modelo LSTM utilizando el conjunto de entrenamiento.  
El proceso incluye:
- Forward y backward pass por época.
- Optimización con Adam.
- Cálculo del error cuadrático medio (MSE) como función de pérdida.

Durante el entrenamiento, se registra la pérdida tanto de entrenamiento como de validación para analizar la convergencia.


In [13]:
from tqdm.auto import tqdm

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30  # puedes dejar 15 si solo haces pruebas
patience = 5     # early stopping: detiene si no mejora en 5 épocas
best_val = float("inf")
wait = 0

train_losses, test_losses = [], []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    
    # Barra de progreso durante el entrenamiento
    progress_bar = tqdm(train_loader, desc=f"Época {epoch+1}/{num_epochs}", leave=False)
    
    for X_batch, y_batch in progress_bar:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
        
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=f"{loss.item():.4f}")
    
    # Promedio de pérdida por época
    train_loss = epoch_loss / len(train_loader)
    train_losses.append(train_loss)
    
    # Validación
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for X_val, y_val in test_loader:
            X_val, y_val = X_val.to(device), y_val.to(device).unsqueeze(1)
            y_pred = model(X_val)
            val_loss += criterion(y_pred, y_val).item()
    val_loss /= len(test_loader)
    test_losses.append(val_loss)
    
    # Mostrar resumen de la época
    print(f"Época [{epoch+1}/{num_epochs}] 🧠 Train: {train_loss:.4f} | Val: {val_loss:.4f}")
    
    # --- EARLY STOPPING ---
    if val_loss < best_val - 1e-5:
        best_val = val_loss
        wait = 0
        torch.save(model.state_dict(), "lstm_best.pt")  # guarda el mejor modelo
    else:
        wait += 1
        if wait >= patience:
            print(f"⏹️ Early stopping en época {epoch+1} — no mejora en {patience} épocas.")
            break
    
    # Guardar checkpoint cada 5 épocas
    if (epoch + 1) % 5 == 0:
        torch.save(model.state_dict(), f"checkpoint_epoch_{epoch+1}.pt")


Época 1/30:   0%|          | 0/1557 [00:00<?, ?it/s]c:\Users\arian\Documents\ruido-cuenca\venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Época [1/30] 🧠 Train: 0.0020 | Val: 0.0001


Época [2/30] 🧠 Train: 0.0002 | Val: 0.0001


Época [3/30] 🧠 Train: 0.0001 | Val: 0.0001


Época [4/30] 🧠 Train: 0.0001 | Val: 0.0000


...

Época [8/30] 🧠 Train: 0.0000 | Val: 0.0000


Época [9/30] 🧠 Train: 0.0000 | Val: 0.0000


Época [10/30] 🧠 Train: 0.0000 | Val: 0.0000


Época [11/30] 🧠 Train: 0.0000 | Val: 0.0000


Época [12/30] 🧠 Train: 0.0000 | Val: 0.0000
⏹️ Early stopping en época 12 — no mejora en 5 épocas.


## Evaluación del modelo

Se evalúa el desempeño del modelo sobre el conjunto de prueba.

Métricas utilizadas:
- **RMSE (Root Mean Square Error)**: mide la desviación media de las predicciones.
- **MAE (Mean Absolute Error)**: mide el error promedio absoluto.
- **R² (Coeficiente de determinación)**: indica qué tan bien el modelo explica la variabilidad de los datos.

También se desnormalizan las predicciones para interpretarlas en unidades reales (dB).


In [14]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device)).cpu().numpy()
    y_true = y_test.numpy()

# Desnormalizar si quieres comparar en valores reales
y_pred_rescaled = scaler.inverse_transform(np.hstack((y_pred, np.zeros((len(y_pred), len(features)-1)))))[:,0]
y_true_rescaled = scaler.inverse_transform(np.hstack((y_true.reshape(-1,1), np.zeros((len(y_true), len(features)-1)))))[:,0]

rmse = np.sqrt(mean_squared_error(y_true_rescaled, y_pred_rescaled))
mae = mean_absolute_error(y_true_rescaled, y_pred_rescaled)
r2 = r2_score(y_true_rescaled, y_pred_rescaled)

print(f"RMSE: {rmse:.3f}")
print(f"MAE : {mae:.3f}")
print(f"R²  : {r2:.3f}")


RMSE: 0.125
MAE : 0.075
R²  : 0.999


## Visualización de resultados

Se comparan las predicciones del modelo LSTM con los valores reales de ruido a lo largo del tiempo.

El objetivo es analizar:
- Qué tan bien el modelo sigue las variaciones reales del ruido.
- Si existen rezagos o desviaciones sistemáticas.

Se grafica la serie temporal de valores **reales vs. predichos** para una muestra representativa.


In [ ]:
plt.figure(figsize=(12,5))
plt.plot(y_true_rescaled[:200], label="Real", linewidth=2)
plt.plot(y_pred_rescaled[:200], label="Predicho", linewidth=2)
plt.title("Predicción de ruido con LSTM")
plt.xlabel("Tiempo")
plt.ylabel("Nivel de ruido (dB)")
plt.legend()
plt.show()
